# Load Senzing truth-sets

These instructions load the [Senzing truth-sets] into the Senzing engine.

In this exercise:

1. [Senzing truth-sets] are downloaded from the internet.
1. Data source names are extracted from the downloaded data.
1. The data source names are added to the Senzing configuration.
1. Records from the downloaded data are added to Senzing.
1. Example queries are demonstrated.

**Warning**:
Remember that the notebooks are not permanent when using the Playground docker container. 
You can save a notebook to your workstation by selecting <b>File</b> > <b>Download</b> in Jupyter Lab.

[Senzing truth-sets]: https://github.com/Senzing/truth-sets

## Prepare Go enviroment

Create a function for creating the Senzing AbstractFactory.

In [ ]:
import (
	"google.golang.org/grpc"
	"google.golang.org/grpc/credentials/insecure"
    "github.com/senzing-garage/sz-sdk-go-grpc/szabstractfactory"
    "github.com/senzing-garage/sz-sdk-go/senzing"    
)

type DataSourceKey struct {
    Data_Source      string
}

var (
	grpcAddress    = "localhost:8261"
    jsonDataSource DataSourceKey
)   

In [ ]:
func testErr(err error) {
    if err != nil {
        panic(err)
    }
}


In [ ]:
func getSzAbstractFactory() senzing.SzAbstractFactory {
    grpcConnection, err := grpc.NewClient(grpcAddress, grpc.WithTransportCredentials(insecure.NewCredentials()))
    testErr(err)
    return &szabstractfactory.Szabstractfactory{
    	GrpcConnection: grpcConnection,
    }    
} 

In [ ]:
func getTruthSetFilenames() []string {
    return []string{
    "customers.json",
    "reference.json",
    "watchlist.json",
    }
}    

In [ ]:
func downloadFile(url string, filepath string) error {
	outputFile, err := os.Create(filepath)
	if err != nil {
		return err
	}
	defer outputFile.Close()

	response, err := http.Get(url)
	if err != nil {
		return err
	}
	defer response.Body.Close()

	_, err = io.Copy(outputFile, response.Body)
	if err != nil {
		return err
	}

	return nil
}

Set environment variables for downloading truth-set files.

In [ ]:
%%
homePath := "./"
truthSetURLPrefix := "https://raw.githubusercontent.com/Senzing/truth-sets/refs/heads/main/truthsets/demo/"
truthSetFilenames := getTruthSetFilenames()

for i := 0; i < len(truthSetFilenames); i++ {
    url := fmt.Sprintf("%s/%s", truthSetURLPrefix, truthSetFilenames[i])
    filepath := fmt.Sprintf("%s%s", homePath, truthSetFilenames[i])
    err := downloadFile(url, filepath)
    testErr(err)
}

Download truth-set files.

## Identify data sources

Discover `DATA_SOURCE` values in records.

In [ ]:
%%
homePath := "./"
truthSetFilenames := getTruthSetFilenames()

result := []string{}

for i := 0; i < len(truthSetFilenames); i++ {
    filepath := fmt.Sprintf("%s%s", homePath, truthSetFilenames[i])
    file, err := os.Open(filepath)
    if err != nil {
        panic(err)
    }
    defer file.Close()   


    scanner := bufio.NewScanner(file)
    for scanner.Scan() {
        line := scanner.Bytes()
        err := json.Unmarshal(line, &jsonDataSource)  
        testErr(err)
        fmt.Printf(">>> %s\n", jsonDataSource.Data_Source)
        if !slices.Contains(result, jsonDataSource.Data_Source) {
            fmt.Printf(">>>>> Adding: %s\n", jsonDataSource.Data_Source)
            result = append(result, jsonDataSource.Data_Source) 
        }        
    }


    if err := scanner.Err(); err != nil {
        log.Fatal(err)
    }    
}
fmt.Printf(">>>>>X %v\n", result)


## Update Senzing configuration

Create an [abstract factory] for accessing Senzing via gRPC.

[abstract factory]: https://en.wikipedia.org/wiki/Abstract_factory_pattern

Using the abstract factory, create Senzing objects.

Get current Senzing configuration.

Add DataSources to Senzing configuration.

Persist new Senzing configuration.

With the change in Senzing configuration, Senzing objects need to be updated.

## Add records

Print help for `sz_engine.add_record()` to see parameters.

Call Senzing repeatedly to add records.

## View results

Retrieve an entity by identifying a record of the entity. Use the `SZ_ENTITY_INCLUDE_RECORD_SUMMARY` flag from among the [get_entity flags].

[get_entity flags]: https://senzing.com/docs/4_beta/flags/flags_get_entity/

Search for entities by attributes.